In [19]:
import random,math,timeit
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy as dc
from sklearn import svm
from sklearn import model_selection as ms

In [20]:
np.random.seed(14)

In [21]:
def random_search(n,dim):
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens



In [22]:
def case1(move):
    return 1 if random.uniform(-0.1,0.9)<move else 0
def case2(one_bin):
    if random.uniform(-0.1,0.9)<math.tanh(int(one_bin)):
        if one_bin==1:
            return 0
        else:return 1
    else:return one_bin
def exchange_binary(binary,score,alpha,beta,gamma):
    al_binary=binary
    e=0.05*random.uniform(0,1)
    movement=beta*math.exp(-gamma*score**2)*score + alpha*e
    if random.uniform(0,1) < movement:
        for i,b in enumerate(binary):
            move=beta*math.exp(-gamma*b**2)*b + alpha*e
            al_binary[i]=case1(move)
    else:
        for i,b in enumerate(binary):
            move=beta*math.exp(-gamma*b**2)*b + alpha*e
            al_binary[i]=case2(move)
    return al_binary

In [24]:
def evaluate(train_d,train_l,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in train_d])
        kf = ms.StratifiedKFold(n_splits=4)
        s = 0
        for tr_ix,te_ix in kf.split(al_data, train_l):
            s+= svm.LinearSVC().fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])#.predict(al_test_data)
        s/=4
        return s

In [25]:
def BFA(train_d,train_l,n=20,max_iter=25,gamma=0.20,beta=0.20,alpha=0.25):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            max_iter: Number of max iteration, default=300
            }
    output:{
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    dim=len(train_d[0])
    global_best=float("-inf")
    global_position=tuple([0]*dim)
    gens_dict = {tuple([0]*dim):float("-inf")}
    gens=random_search(n,dim)
    for gen in gens:
        if tuple(gen) in gens_dict:
            score = gens_dict[tuple(gen)]
        else:
            score=evaluate(train_d,train_l,gen)
            gens_dict[tuple(gen)]=score
        if score > global_best:
            global_best=score
            global_position=dc(gen)
    for it in range(max_iter):
        for i,x in enumerate(gens):
            for j,y in enumerate(gens):
                if gens_dict[tuple(y)] < gens_dict[tuple(x)]:
                    gens[j]=exchange_binary(y,gens_dict[tuple(y)],alpha,beta,gamma)
                gen = gens[j]
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gens[j])
                    gens_dict[tuple(gen)]=score
                if score > global_best:
                    global_best=score
                    global_position=dc(gen)
    return global_position,global_position.count(1)


In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
import numpy as np

def test_score(gen, tr_x, tr_y, te_x, te_y):
    mask = np.array(gen) == 1
    al_data = np.array(tr_x[:, mask])
    al_test_data = np.array(te_x[:, mask])

    accuracies, precisions, recalls, f1s = [], [], [], []
    for _ in range(5):
        model = SVC(kernel='rbf', C=1.0, gamma='scale') 
        model.fit(al_data, tr_y)
        preds = model.predict(al_test_data)

        accuracies.append(accuracy_score(te_y, preds))
        precisions.append(precision_score(te_y, preds, zero_division=0))
        recalls.append(recall_score(te_y, preds, zero_division=0))
        f1s.append(f1_score(te_y, preds, zero_division=0))

    return (
        np.mean(accuracies),
        np.mean(precisions),
        np.mean(recalls),
        np.mean(f1s)
    )

In [27]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

with open("../../data/wdbc.data.csv") as f:
    lines = f.read().splitlines()

# id (columna 0), diagnosis (columna 1), resto son features numéricas
x = np.array([[float(d) for d in line.split(',')[2:]] for line in lines])  # desde la 3ra columna
y = np.array([line.split(',')[1] for line in lines])  # diagnosis ('M' o 'B')

lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)

train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)


print("Shape X:", x.shape)

Shape X: (569, 30)


In [28]:
# Evaluación inicial con todas las características
k = [1 for r in range(len(x[0]))]
acc, prec, rec, f1 = test_score(k, train_d, train_l, test_d, test_l)
print(f"All features → Acc:{acc:.4f}  Prec:{prec:.4f}  Rec:{rec:.4f}  F1:{f1:.4f}")

# Inicialización de métricas
fattr = 0
ftest_acc = 0.0
ftest_prec = 0.0
ftest_rec = 0.0
ftest_f1 = 0.0

flist = [0 for _ in range(len(x[0]))]
final_list = [0 for _ in range(len(x[0]))]

start = timeit.default_timer()

# Repeticiones
for i in range(20):
    g, l = BFA(train_d, train_l, n=10, max_iter=25, gamma=0.20, beta=0.20, alpha=0.25)
    fattr += l

    acc, prec, rec, f1 = test_score(g, train_d, train_l, test_d, test_l)
    ftest_acc += acc
    ftest_prec += prec
    ftest_rec += rec
    ftest_f1 += f1

    for j in range(len(flist)):
        if g[j] == 1:
            flist[j] += 1

    print(f"{i+1:02d}  {''.join(map(str,g))}  {l}  Acc:{acc:.4f}  F1:{f1:.4f}")

# Promedios finales
fattr /= 20
ftest_acc /= 20
ftest_prec /= 20
ftest_rec /= 20
ftest_f1 /= 20

end = timeit.default_timer()
time = end - start

# Selección final de características
final_count = int(round(fattr))
final_count = max(0, min(final_count, len(flist)))
final = np.argsort(flist)[::-1][:final_count]
for i in range(len(final)):
    final_list[final[i]] = 1

print(f"\nFinal: {''.join(map(str,final_list))}")
print(f"Selected features: {fattr:.2f}")
print(f"Average metrics → Acc:{ftest_acc:.4f}  Prec:{ftest_prec:.4f}  Rec:{ftest_rec:.4f}  F1:{ftest_f1:.4f}")
print(f"Execution time: {time:.4f} s")

All features → Acc:0.9371  Prec:1.0000  Rec:0.8302  F1:0.9072
01  000110100110001011111100101110  16  Acc:0.8881  F1:0.8261
02  110001011010100111101110001001  16  Acc:0.9510  F1:0.9293
03  000111111100100111111011100110  19  Acc:0.9371  F1:0.9072
04  111111111011101111111110111101  26  Acc:0.8951  F1:0.8387
05  111110111110101101111011111100  23  Acc:0.9371  F1:0.9072
06  010000111000000000101000000100  7  Acc:0.9231  F1:0.8866
07  111111110101010000001001111101  18  Acc:0.9371  F1:0.9072
08  110010110000110001011000000101  12  Acc:0.9441  F1:0.9184
09  110101101111100101011110110101  20  Acc:0.8951  F1:0.8387
10  111111111111101111111111110111  28  Acc:0.9301  F1:0.8958
11  010000000000000000001000000110  4  Acc:0.9301  F1:0.9000
12  101111011001011001011010001110  17  Acc:0.9021  F1:0.8478
13  101101111111010011101011101111  22  Acc:0.9371  F1:0.9072
14  111100110110011101100101100101  18  Acc:0.9371  F1:0.9072
15  111000011010010000010101000001  11  Acc:0.9301  F1:0.8980
16  000000